In [1]:
# import libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import streamlit as st
import datetime
from datetime import datetime

In [2]:
#define functions

In [3]:
# load data
@st.cache_data # a way to reloadthe streamlit once
def load_data():
    
    df_agg = pd.read_csv('Aggregated_Metrics_By_Video.csv').iloc[1:,:] #skipp the first row
    df_agg.columns = ['Video','Video title','Video publish time','Comments added','Shares','Dislikes','Likes',
                      'Subscribers lost','Subscribers gained','RPM(USD)','CPM(USD)','Average % viewed','Average view duration',
                      'Views','Watch time (hours)','Subscribers','Your estimated revenue (USD)','Impressions','Impressions ctr(%)']
    df_agg['Video publish time'] = pd.to_datetime(df_agg['Video publish time'])
    df_agg['Average view duration'] = df_agg['Average view duration'].apply(lambda x: datetime.strptime(x,'%H:%M:%S'))
    df_agg['Avg_duration_sec'] = df_agg['Average view duration'].apply(lambda x: x.second + x.minute*60 + x.hour*3600)
    df_agg['Engagement_ratio'] =  (df_agg['Comments added'] + df_agg['Shares'] +df_agg['Dislikes'] + df_agg['Likes']) /df_agg.Views
    df_agg['Views / sub gained'] = df_agg['Views'] / df_agg['Subscribers gained']
    df_agg.sort_values('Video publish time', ascending = False, inplace = True)    
    df_agg_sub = pd.read_csv('Aggregated_Metrics_By_Country_And_Subscriber_Status.csv')
    df_comments = pd.read_csv('Aggregated_Metrics_By_Video.csv')
    df_time = pd.read_csv('Video_Performance_Over_Time.csv')
    df_time['Date'] = pd.to_datetime(df_time['Date'])
    return df_agg, df_agg_sub, df_comments, df_time 

2023-03-28 16:21:24.889 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [4]:
df_agg, df_agg_sub, df_comments, df_time = load_data()

2023-03-28 16:21:30.285 
  command:

    streamlit run c:\Users\kelida\Desktop\streamlit\stenv\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2023-03-28 16:21:30.292 No runtime found, using MemoryCacheStorageManager


In [5]:
# buidl a dashboard
add_sidebar = st.sidebar.selectbox('Aggreggate or Individual Video', ('Aggregate Metrics', 'Individual Video Analysis'))


In [6]:
df_agg_diff = df_agg.copy()

In [7]:
df_agg_diff.head()

,Video,Video title,Video publish time,Comments added,Shares,Dislikes,Likes,Subscribers lost,Subscribers gained,RPM(USD),...,Average view duration,Views,Watch time (hours),Subscribers,Your estimated revenue (USD),Impressions,Impressions ctr(%),Avg_duration_sec,Engagement_ratio,Views / sub gained
111,0jTtHYie3CU,Should You Be Excited About Web 3? (As a Data ...,2022-01-17,37,43,8,267,14,18,4.055,...,1900-01-01 00:02:38,4383,192.5779,4,16.549,65130,2.95,158,0.080995,243.500000
187,2RWwN5ZT4tA,Should @Luke Barousse Take This Data Analyst ...,2022-01-14,12,2,3,78,1,1,1.882,...,1900-01-01 00:00:38,2401,25.9375,0,1.720,25094,2.64,38,0.039567,2401.000000
64,rEWPqw6rMGI,The Only Data Science Explanation You Need,2022-01-10,62,141,5,722,28,136,5.971,...,1900-01-01 00:04:40,10277,801.5549,108,60.498,215491,2.22,280,0.090493,75.566176
59,o-wsyxWbPOw,We Need to Talk About The LinkedIn Machine Lea...,2022-01-03,65,36,12,592,10,78,5.321,...,1900-01-01 00:02:46,11808,545.6332,68,62.568,166915,3.32,166,0.059705,151.384615
32,xpIFS6jZbe8,How I Would Learn Data Science in 2022 (If I H...,2021-12-27,109,767,53,4413,46,2553,6.836,...,1900-01-01 00:04:29,79283,5945.5420,2507,528.286,1420968,3.31,269,0.067379,31.054837


In [8]:

df_agg_diff.columns

Index(['Video', 'Video title', 'Video publish time', 'Comments added',
       'Shares', 'Dislikes', 'Likes', 'Subscribers lost', 'Subscribers gained',
       'RPM(USD)', 'CPM(USD)', 'Average % viewed', 'Average view duration',
       'Views', 'Watch time (hours)', 'Subscribers',
       'Your estimated revenue (USD)', 'Impressions', 'Impressions ctr(%)',
       'Avg_duration_sec', 'Engagement_ratio', 'Views / sub gained'],
      dtype='object')

In [9]:
df_agg_diff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 223 entries, 111 to 223
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Video                         223 non-null    object        
 1   Video title                   223 non-null    object        
 2   Video publish time            223 non-null    datetime64[ns]
 3   Comments added                223 non-null    int64         
 4   Shares                        223 non-null    int64         
 5   Dislikes                      223 non-null    int64         
 6   Likes                         223 non-null    int64         
 7   Subscribers lost              223 non-null    int64         
 8   Subscribers gained            223 non-null    int64         
 9   RPM(USD)                      223 non-null    float64       
 10  CPM(USD)                      221 non-null    float64       
 11  Average % viewed              

In [10]:
metric_data_12mo = df_agg_diff['Video publish time'].max() - pd.DateOffset(months=12)
median_agg = df_agg_diff[df_agg_diff['Video publish time'] >= metric_data_12mo].median()
print(median_agg)

Comments added                      43.500000
Shares                              42.500000
Dislikes                             5.000000
Likes                              382.500000
Subscribers lost                    13.000000
Subscribers gained                  56.500000
RPM(USD)                             4.370000
CPM(USD)                            10.573000
Average % viewed                    41.175000
Views                             7417.000000
Watch time (hours)                 279.985100
Subscribers                         38.500000
Your estimated revenue (USD)        24.799500
Impressions                     155102.500000
Impressions ctr(%)                   2.430000
Avg_duration_sec                   166.000000
Engagement_ratio                     0.060285
Views / sub gained                 140.146406
dtype: float64


C:\Users\kelida\AppData\Local\Temp\ipykernel_9768\2869075216.py:2: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  median_agg = df_agg_diff[df_agg_diff['Video publish time'] >= metric_data_12mo].median()
C:\Users\kelida\AppData\Local\Temp\ipykernel_9768\2869075216.py:2: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  median_agg = df_agg_diff[df_agg_diff['Video publish time'] >= metric_data_12mo].median()


In [11]:
# alternative to the above cell:
numeric_cols = np.array((df_agg_diff.dtypes == 'float64') | (df_agg_diff.dtypes == 'int64'))
print(numeric_cols)
df_agg_diff.iloc[:, numeric_cols] = (df_agg_diff.iloc[:,numeric_cols] - median_agg).div(median_agg)


[False False False  True  True  True  True  True  True  True  True  True
 False  True  True  True  True  True  True  True  True  True]


In [12]:
if add_sidebar == 'Aggregate Metrics':
     #bit of data enginering
     # metrics that are relevant in descending order
     df_agg_metrics = df_agg[['Video publish time','Views','Likes','Subscribers','Shares','Comments added','RPM(USD)','Average % viewed',
                             'Avg_duration_sec', 'Engagement_ratio','Views / sub gained']]
     # figure the most 06/12 month date range 
     metric_data_6mo = df_agg_metrics['Video publish time'].max() - pd.DateOffset(months=6)
     metric_data_12mo = df_agg_metrics['Video publish time'].max() - pd.DateOffset(months=12)
     #take the aggregate of all of the medians
     metric_median6mo = df_agg_metrics[df_agg_metrics['Video publish time'] >= metric_data_6mo].median()
     metric_median12mo = df_agg_metrics[df_agg_metrics['Video publish time'] >= metric_data_12mo].median()

     # instead of metrics we could use columns
     # create 5 columnsin the stramlit dashboard
     col1, col2, col3, col4, col5 = st.columns(5)
     columns = [col1, col2, col3, col4, col5]

     # loop through all the diff columns
     count = 0
     for i in metric_median6mo.index:
          with columns[count]:
               delta = (metric_median6mo[i] - metric_median12mo[i])/metric_median12mo[i]
               st.metric(label= i, value = round(metric_median6mo[i],1), delta = '{:.2%}'.format(delta))
               count += 1
               if count >= 5:
                    count = 0

     # add df to the dashboard
     #get date information / trim to relevant data 
     df_agg_diff['Publish_date'] = df_agg_diff['Video publish time'].apply(lambda x: x.date())
     df_agg_diff_final = df_agg_diff.loc[:,['Video title','Publish_date','Views','Likes','Subscribers','Shares','Comments added','RPM(USD)','Average % viewed',
                             'Avg_duration_sec', 'Engagement_ratio','Views / sub gained']]

C:\Users\kelida\AppData\Local\Temp\ipykernel_9768\1649207867.py:10: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  metric_median6mo = df_agg_metrics[df_agg_metrics['Video publish time'] >= metric_data_6mo].median()
C:\Users\kelida\AppData\Local\Temp\ipykernel_9768\1649207867.py:11: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  metric_median12mo = df_agg_metrics[df_agg_metrics['Video publish time'] >= metric_data_12mo].median()


In [23]:
# merge the time data merge dailly with publish datas to get delta
df_time_diff = pd.merge(df_time, df_agg.loc[:,['Video', 'Video publish time']], left_on = 'External Video ID', right_on='Video')
df_time_diff['days_published'] = (df_time_diff['Date'] - df_time_diff['Video publish time']).dt.days

#get last 12 mo of data rather than all data
date_12mo = df_agg['Video publish time'].max() - pd.DateOffset(months=12)
df_time_diff_yr = df_time_diff[df_time_diff['Video publish time'] >= date_12mo]

In [25]:
#get daily view data(first 30), median & percentiles the below code could be done by groupby 
# but we use pivot tables
views_days = pd.pivot_table(df_time_diff_yr, index='days_published', values='Views', aggfunc=[np.mean,np.median,lambda x: np.percentile(x, 80),lambda x: np.percentile(x, 20)]).reset_index()
views_days.columns = ['days_published','mean_views','median_views','80pct_views','20pct_views']
#take the days between 0 and 30
views_days = views_days[views_days['days_published'].between(0,30)]
views_cumulative = views_days.loc[:,['days_published','median_views','80pct_views','20pct_views']]
views_cumulative.loc[:,['median_views','80pct_views','20pct_views']] = views_cumulative.loc[:,['median_views','80pct_views','20pct_views']].cumsum()

## build dashboard
views_cumulative

,days_published,median_views,80pct_views,20pct_views
4,0,1651.5,4364.0,1106.0
5,1,2407.5,6655.0,1471.0
6,2,2791.5,7841.0,1680.0
7,3,3089.5,8605.0,1827.0
8,4,3281.5,9252.6,1949.4
9,5,3461.5,9789.0,2051.2
10,6,3619.5,10180.4,2131.8
11,7,3727.0,10488.6,2189.4
12,8,3831.5,10720.8,2234.4
13,9,3925.5,10916.8,2275.0


In [18]:
df_agg_numeric_lst = df_agg_diff_final.median().index.tolist()
print(df_agg_numeric_lst)
df_to_pct = {}
for i in df_agg_numeric_lst:
    df_to_pct[i] = '{:.1%}'.format
df_to_pct    

['Views', 'Likes', 'Subscribers', 'Shares', 'Comments added', 'RPM(USD)', 'Average % viewed', 'Avg_duration_sec', 'Engagement_ratio', 'Views / sub gained']


C:\Users\kelida\AppData\Local\Temp\ipykernel_9768\3612223704.py:1: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_agg_numeric_lst = df_agg_diff_final.median().index.tolist()


{'Views': <function str.format>,
 'Likes': <function str.format>,
 'Subscribers': <function str.format>,
 'Shares': <function str.format>,
 'Comments added': <function str.format>,
 'RPM(USD)': <function str.format>,
 'Average % viewed': <function str.format>,
 'Avg_duration_sec': <function str.format>,
 'Engagement_ratio': <function str.format>,
 'Views / sub gained': <function str.format>}

In [19]:
videos = tuple(df_agg['Video title'])
videos

('Should You Be Excited About Web 3? (As a Data Scientist)',
 'Should  @Luke Barousse Take This Data Analyst Job? (Funny) #Shorts',
 'The Only Data Science Explanation You Need',
 'We Need to Talk About The LinkedIn Machine Learning Assessment.',
 'How I Would Learn Data Science in 2022 (If I Had to Start Over)',
 'What the Heck is WSL 2? (My New Favorite Tool)',
 'How Statistics Saved the US SERIOUS $$$$ During WW2 #Shorts',
 'How Zillow Lost $500 MILLION With Machine Learning',
 'Why Is Data Engineering So HOT Right Now?',
 'Is Data Science Dying?',
 '7 Incredible Books That Transformed My Health and My Life',
 'Why EVERYONE Should Start a Podcast (Including YOU)',
 'Building a Burrito Dashboard - Data Science Project from Scratch with atoti',
 'ML Ops: What is it REALLY?',
 'Business Skills for Data Science: What are they REALLY?',
 '#66DaysOfData Round 3 Live Event! (feat. @StatQuest with Josh Starmer)',
 'The TRUTH About My First Data Science Project',
 "Is Data Visualization Impo

In [13]:
metric_data_6mo

Timestamp('2021-07-17 00:00:00')

In [20]:
metric_median6mo

Views                 6062.000000
Likes                  305.000000
Subscribers             22.000000
Shares                  41.000000
Comments added          36.000000
RPM(USD)                 4.395000
Average % viewed        41.225000
Avg_duration_sec       175.000000
Engagement_ratio         0.067113
Views / sub gained     146.869727
dtype: float64

In [22]:
df_agg.columns

Index(['Video', 'Video title', 'Video publish time', 'Comments added',
       'Shares', 'Dislikes', 'Likes', 'Subscribers lost', 'Subscribers gained',
       'RPM(USD)', 'CPM(USD)', 'Average % viewed', 'Average view duration',
       'Views', 'Watch time (hours)', 'Subscribers',
       'Your estimated revenue (USD)', 'Impressions', 'Impressions ctr(%)',
       'Avg_duration_sec', 'Engagement_ratio', 'Views / sub gained'],
      dtype='object')

In [15]:
df_agg_diff_final

,Video title,Publish_date,Views,Likes,Subscribers,Shares,Comments added,RPM(USD),Average % viewed,Avg_duration_sec,Engagement_ratio,Views / sub gained
111,Should You Be Excited About Web 3? (As a Data ...,2022-01-17,-0.409060,-0.301961,-0.896104,0.011765,-0.149425,-0.072082,0.177899,-0.048193,0.343527,0.737469
187,Should @Luke Barousse Take This Data Analyst ...,2022-01-14,-0.676284,-0.796078,-1.000000,-0.952941,-0.724138,-0.569336,0.716576,-0.771084,-0.343672,16.132084
64,The Only Data Science Explanation You Need,2022-01-10,0.385601,0.887582,1.805195,2.317647,0.425287,0.366362,-0.138069,0.686747,0.501088,-0.460805
59,We Need to Talk About The LinkedIn Machine Lea...,2022-01-03,0.592018,0.547712,0.766234,-0.152941,0.494253,0.217620,0.094839,0.000000,-0.009619,0.080189
32,How I Would Learn Data Science in 2022 (If I H...,2021-12-27,9.689362,10.537255,64.116883,17.047059,1.505747,0.564302,-0.106982,0.620482,0.117670,-0.778411
...,...,...,...,...,...,...,...,...,...,...,...,...
75,Should You Get A Masters in Data Science?,2018-11-14,1.492652,-0.278431,1.051948,-0.035294,0.287356,0.692906,0.098239,-0.084337,-0.656364,0.628632
190,How I Became A Data Scientist From a Business ...,2018-11-12,-0.256438,-0.560784,1.103896,-0.223529,-0.747126,0.011213,-0.069095,0.427711,-0.350323,-0.514176
204,Predicting Season Long NBA Wins Using Multiple...,2018-07-10,-0.074693,-0.584314,-0.142857,0.058824,-0.839080,-0.340275,-0.345962,-0.126506,-0.485180,0.440301
138,Predicting Crypto-Currency Price Using RNN lST...,2017-11-18,1.232439,-0.354248,1.857143,1.682353,-0.356322,-0.696568,-0.172799,-0.367470,-0.592267,0.064395
